This model predicts the makeup of the travel party (adults, children, or mixed).

In [1]:
import os
import toml
import pandas as pd
import validation_data_input
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
tour_data = validation_data_input.get_tours_data()

# all joint tours
joint_tour_data = tour_data.loc[tour_data['tour_category'] == "joint"]

- joint tour count

In [3]:
# total number of joint tours by source
df_1 = joint_tour_data.groupby('source')['trip_weight_2017_2019'].sum().reset_index()
df_1 = df_1.rename(columns={"trip_weight_2017_2019": "joint tour count"})
# total number of tours by source
df_2 = tour_data.groupby('source')['trip_weight_2017_2019'].sum().reset_index()
df_2 = df_2.rename(columns={"trip_weight_2017_2019": "tour count"})

df_joint_tour_count = df_1.merge(df_2, how="left", on='source')
df_joint_tour_count['joint_percent'] = df_joint_tour_count['joint tour count']/df_joint_tour_count['tour count']
df_joint_tour_count['tour count'] = df_joint_tour_count['tour count'].astype(float).map(lambda n: '{:.2f}'.format(n))
df_joint_tour_count['joint tour count'] = df_joint_tour_count['joint tour count'].astype(float).map(lambda n: '{:.2f}'.format(n))
df_joint_tour_count['joint_percent'] = df_joint_tour_count['joint_percent'].astype(float).map(lambda n: '{:.2%}'.format(n))
df_joint_tour_count

,source,joint tour count,tour count,joint_percent
0,model results,107232.00,4812447.00,2.23%
1,survey data,316562.43,3362433.66,9.41%
2,unweighted survey,706.00,14072.00,5.02%


In [4]:
df_plot = joint_tour_data.groupby(['source','composition'])[['trip_weight_2017_2019']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
    apply(lambda x: x / float(x.sum()))
# df_plot
fig = px.bar(df_plot, x="composition", y="percentage", color="source",barmode="group", template="simple_white",
             # color_discrete_sequence=config['psrc_color'],
             title="joint tour composition")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis=dict(tickformat=".2%"), xaxis=dict(dtick="1"))
fig.show()